# Similarity Queries

In [0]:
from pprint import pprint
from collections import defaultdict
from gensim import corpora, models, similarities

### **Creating the Corpus**

In [0]:
documents = ["Human machine interface for lab abc computer applications",
              "A survey of user opinion of computer system response time",
              "The EPS user interface management system",
              "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

In [3]:
# remove common words and tokenize
stopwords = set('for a of the and to in'.split())

texts = [ [ word for word in document.lower().split() if word not in stopwords ] for document in documents ]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1

texts = [ [token for token in text if frequency[token] > 1] for text in texts ]

# Map each word in corpus to a unique integer ID
dictionary = corpora.Dictionary(texts)

# Build corpus with BoW representation
corpus = [ dictionary.doc2bow(text) for text in texts ]

pprint(corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


### **Similarity interface**

In [0]:
# Initialize a 2-dimensioanl LSI transformation
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [5]:
# Create a 'query document' and transform it to a BoW representation
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
print(vec_bow)

[(0, 1), (1, 1)]


In [6]:
# Convert query to 2-D LSI space
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 0.4618210045327156), (1, -0.07002766527900048)]


***Initializing query structures***

In [7]:
# Build index of document cosine similarities
index = similarities.MatrixSimilarity(lsi[corpus])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [8]:
# Save and load for index persistency
index.save('/deerwester.index')
index = similarities.MatrixSimilarity.load('/deerwester.index')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


***Performing queries***

In [9]:
# Perform a similarity query of the 'query document' against the corpus
sims = index[vec_lsi]

# Sort in descending order of similarities
sims = sorted(enumerate(sims), key=lambda item : -item[1])

for i, s in enumerate(sims):
  print(s, documents[i])

(2, 0.9984453) Human machine interface for lab abc computer applications
(0, 0.998093) A survey of user opinion of computer system response time
(3, 0.9865886) The EPS user interface management system
(1, 0.93748635) System and human system engineering testing of EPS
(4, 0.90755945) Relation of user perceived response time to error measurement
(8, 0.050041765) The generation of random binary unordered trees
(7, -0.09879464) The intersection graph of paths in trees
(6, -0.10639259) Graph minors IV Widths of trees and well quasi ordering
(5, -0.12416792) Graph minors A survey
